Get the data. Google blocks auto-download, so it is saved on github.

In [1]:
from urllib.request import urlopen
import pandas as pd

Data_URL = 'https://raw.githubusercontent.com/b-k/large-files/master/Pedido_195_2020.csv'

def get_data():    
    """Download a copy of the survey if we don't already have it. Return a data frame with the observations."""
    try:
        return pd.read_csv(open("survey.csv", 'rb'), sep=';')
    except FileNotFoundError:
        in_csv = urlopen(Data_URL).read().decode('ISO-8859-1')
        f = open("survey.csv", 'w')                      
        for data in in_csv:                              
            f.write(data) #Seems like a bug that commas are still treated as separators?
#            f.write(data.replace(',','-')) #Seems like a bug that commas are still treated as separators?
        
        return pd.read_csv(open("survey.csv", 'rb'), delimiter=';')

In [2]:
d=get_data()

Define some convenience functions:

In [3]:
# Use Python's dictionary structure for an actual dictionary.
# I didn't translate things that seemed like easy cognates like "Interior de taxi" or "Estabelecimento comercial"
p2e_d = {"Floresta": "Forest",
      'Sem informação': "No information",
        'Estação de barcas': "Ferry",
         'Estação ferroviária': 'Train station',
         'Estação rodoviária': 'Bus station (formal)',
         'Interior de coletivo': 'Bus station (informal)',
         'Estab. público estadual': "Local government building",
         'Estabelecimento de ensino': 'school',
         'Morro': 'Hill(?)',
         'Praia': 'Beach',
         'Clube/instituição desportiva' : 'Sports club',
         'Mar': 'Sea',
         'Rio': 'River',
         'Lagoa': 'Lagoon',
         'Campo': 'Country',
         'Boate': 'Night club',
         'Caixa Eletrônico': 'ATM',
         'Conjunto habitacional': 'Apartment',
         'Semáforo': 'Stoplight'
        }

def p2e(p_in):
    try:
        return p2e_d[p_in]
    except:
        return p_in

In [4]:
def get_by_sex(xtab, location, sex):
    try:
        return locale_sex.loc[(location, sex)]
    except:
        return 0
    
# Python isn't IEEE 754 compliant, so we have to write our own division function.
def ratio_754(a,b):
    return a/b if b > 0 else float("nan") if a==0 else float("inf") if a>0 else -float("inf")

Get some aggregations via the SQL analogue groupby, which listwise deletes NaN.

Then put some useful things into a single tuple:  
[locale, percent female, count (with sex), average age]

In [5]:
locale_sex = d.groupby(["local", "sexo"])["controle"].count()
locale_age =d.groupby(["local"])["idade"].mean()
locales = set([t[0] for t in locale_sex.index])

In [6]:
m_and_f = lambda L: (get_by_sex(locale_sex, L, 'feminino')+get_by_sex(locale_sex, L, 'masculino'))
pct_f = [(L, ratio_754(get_by_sex(locale_sex, L, 'feminino'), m_and_f(L)), m_and_f(L), locale_age.loc[L]) for L in locales]

Output: sort locations by percent female, and by average age.

In [7]:
pct_f.sort(key=lambda x:-x[1]+0.0)
for i in pct_f:
    print (f'{i[1]:.1%}: {p2e(i[0])} ({i[2]})')

100.0%: Estab. público federal (1)
53.8%: Estabelecimento prisional (987)
50.0%: Ferry (4)
43.2%: Local government building (118)
33.3%: Casa de show (6)
30.2%: Bus station (formal) (116)
28.6%: Interior de taxi (35)
25.1%: Bus station (informal) (573)
25.0%: Aeroporto (8)
25.0%: Instituição religiosa (12)
20.0%: Motel (20)
19.4%: Interior de transporte alternativo (31)
19.2%: Estab. público municipal (26)
16.8%: Bar/restaurante (345)
16.7%: school (42)
16.7%: Supermercado (6)
15.8%: Stoplight (19)
13.6%: No information (140)
13.1%: Estabelecimento comercial (464)
12.9%: Residência (17013)
12.5%: Estádio (8)
11.9%: Posto de gasolina (42)
11.1%: Interior de composição férrea (9)
10.7%: Hospital, clínica e similares (28)
10.7%: Vila (84)
9.2%: Outros (4538)
8.5%: Estabelecimento hoteleiro (47)
8.3%: Ignorado (4043)
8.2%: Train station (85)
8.0%: Teatro (25)
7.7%: Parque (26)
7.7%: Delegacia policial (26)
7.5%: Apartment (146)
6.7%: Condomínio (254)
6.5%: Beach (340)
6.2%: Night club (16)

In [8]:
pct_f.sort(key=lambda x:-x[3]+0.0)
for i in pct_f:
    print (f'{int(i[3]) if i[3]==i[3] else 0}: {p2e(i[0])} ({i[2]})')

58: Estação metroviária (1)
46: Chácara (1)
35: Estabelecimento militar (2)
34: Estabelecimento financeiro (3)
32: Casa de show (6)
30: Sítio (85)
29: Sports club (9)
29: Estabelecimento prisional (987)
29: Estabelecimento comercial (464)
29: Bar/restaurante (345)
28: Hospital, clínica e similares (28)
28: Supermercado (6)
27: Delegacia policial (26)
27: Local government building (118)
26: Shopping center (10)
26: Escritório (7)
26: Estabelecimento hoteleiro (47)
26: No information (140)
25: Garagem (2)
25: Estacionamento (14)
25: Fazenda (45)
24: Residência (17013)
24: Instituição religiosa (12)
24: Posto de gasolina (42)
24: Vila (84)
23: Bus station (informal) (573)
23: Estab. público municipal (26)
23: Bus station (formal) (116)
23: Motel (20)
23: Interior de embarcação (5)
23: Ignorado (4043)
23: Linha férrea (56)
23: River (9)
22: Interior de taxi (35)
22: Condomínio (254)
22: Aeroporto (8)
22: Outros (4538)
22: ATM (2)
22: Stoplight (19)
22: Via pública (55584)
22: Sea (9)
22: I